# Section 1: Introduction

## Purpose: 

This notebook implements a function called get_data() using the yfinance package to download hourly stock data for the five FAANG companies over the past five days. The data is saved in a timestamped CSV file within a data/ folder.

**FAANG Stocks:**

- Facebook → META # Facebook has rebranded to [Meta Platforms, Inc.](https://about.fb.com/news/2021/10/facebook-company-is-now-meta/)

- Apple → AAPL # [Apple Inc.](https://www.apple.com)

- Amazon → AMZN # [Amazon.com, Inc.](https://www.amazon.com)

- Netflix → NFLX # [Netflix, Inc.](https://www.netflix.com)

- Google → GOOG # [Alphabet Inc.](https://www.google.com)

# Section 2: Import Required Libraries

In [ ]:
# Yahoo Finance API
import yfinance as yf
# Pandas is used for data manipulation
import pandas as pd
# OS Module is used for file path operations
import os
# Datetime is used for handling date and time
from datetime import datetime, timedelta

### Explanation of Libraries Used:

- **`yfinance`**: Used to fetch stock data from Yahoo Finance. [Source](https://pypi.org/project/yfinance/).  

- **`pandas`**: For handling tabular data and saving to CSV. [Source](https://pandas.pydata.org/)

- **`os`**: Used to check and create directories.

- **`datetime`**: Used to generate timestamped filenames.

### yfinance GitHub Repository

The [`yfinance`](https://github.com/ranaroussi/yfinance) project is an open-source Python library that provides a convenient way to download financial and market data from Yahoo Finance.

- Developed by **Ran Aroussi**, it offers a Pythonic interface to access stock prices, historical data, and financial metrics.
- It supports features like:
  - `Ticker`: Retrieve data for a single stock
  - `download`: Bulk download for multiple tickers
  - `Sector`, `Industry`, and `Screener`: For market analysis
  - `WebSocket`: For live streaming data

> ⚠️ Note: yfinance is not affiliated with or endorsed by Yahoo Inc. It uses publicly available APIs and is intended for **research and educational purposes only** which is the purpose for this notebook.

For installation:
```bash
pip install yfinance

[Source](https://pypi.org/project/yfinance/)